In [48]:
import dash
from dash import dcc, html
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

app = dash.Dash(__name__)

# Load LSTM models for BTC, ETH, ADA
models = {
    "BTC": load_model("models/BTC_lstm_model.h5"),
    "ETH": load_model("models/ETH_lstm_model.h5"),
    "ADA": load_model("models/ADA_lstm_model.h5"),
}


def handle_data(currency, look_back=60, train_data_percent=0.9):
    df = pd.read_csv(f"data/{currency}-USD.csv")
    df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
    df.set_index("Date", inplace=True)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[["Close"]])

    # Calculate the split index
    split_idx = int(len(scaled_data) * train_data_percent)

    # Create data structure with look_back timesteps and 1 output
    X, y = [], []
    for i in range(look_back, len(scaled_data)):
        X.append(scaled_data[i - look_back : i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)

    # Split the data into training and test sets
    X_train, X_test = X[:split_idx], X[split_idx:]

    # Reshape X to be [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    return X_test, df, scaler


def process_data(currency):
    look_back = 60
    X_test, df, scaler = handle_data(currency, look_back)
    predictions = models[currency].predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    df = df.iloc[look_back:]  # Adjust for look_back period
    df["Predictions"] = np.nan
    df.iloc[-len(predictions) :, df.columns.get_loc("Predictions")] = (
        predictions.flatten()
    )

    return df


data = {
    "BTC": process_data("BTC"),
    "ETH": process_data("ETH"),
    "ADA": process_data("ADA"),
}

app.layout = html.Div(
    [
        html.H1(
            "Cryptocurrency Price Analysis Dashboard", style={"textAlign": "center"}
        ),
        dcc.Tabs(
            id="tabs",
            children=[
                dcc.Tab(
                    label=f"{currency}-USD Data",
                    children=[
                        html.Div(
                            [
                                html.H2(
                                    f"{currency} LSTM Predicted Closing Price",
                                    style={"textAlign": "center"},
                                ),
                                dcc.Graph(id=f"{currency.lower()}-graph"),
                            ]
                        )
                    ],
                )
                for currency in ["BTC", "ETH", "ADA"]
            ],
        ),
    ]
)


@app.callback(
    [
        Output(f"{currency.lower()}-graph", "figure")
        for currency in ["BTC", "ETH", "ADA"]
    ],
    [Input("tabs", "value")],
)
def update_graphs(_):
    figures = []
    for currency in ["BTC", "ETH", "ADA"]:
        df = data[currency]
        valid_df = df.dropna(subset=["Predictions"])

        figures.append(
            {
                "data": [
                    go.Scatter(x=valid_df.index, y=valid_df["Predictions"], name="Predicted Close", mode="text", text="-", textfont=dict(size=25, color='blue')),
                    go.Candlestick(
                        x=valid_df.index,
                        open=valid_df["Open"],
                        high=valid_df["High"],
                        low=valid_df["Low"],
                        close=valid_df["Close"],
                        name="Candlestick",
                    )
                ],
                "layout": go.Layout(
                    title=f"{currency} Closing Price Prediction",
                    xaxis={"title": "Date"},
                    yaxis={"title": "Price"},
                ),
            }
        )
    return figures


if __name__ == "__main__":
    app.run_server(debug=True)

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 565ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 503ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 472ms/step
